In [1]:
import plotly.express as px
import pandas as pd
import numpy as np

In [7]:
# US state abbreviations
states = [
    'AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA',
    'HI','ID','IL','IN','IA','KS','KY','LA','ME','MD',
    'MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ',
    'NM','NY','NC','ND','OH','OK','OR','PA','RI','SC',
    'SD','TN','TX','UT','VT','VA','WA','WV','WI','WY'
]

years = list(range(2010, 2025))

# Generate base prices per state, then add yearly growth + noise
np.random.seed(42)
base_prices = np.random.randint(150000, 500000, size=len(states))

rows = []
for i, state in enumerate(states):
    for year in years:
        # ~3-5% annual growth + random noise
        growth = (1 + np.random.uniform(0.03, 0.05)) ** (year - 2010)
        noise = np.random.normal(0, 10000)
        price = int(base_prices[i] * growth + noise)
        rows.append({'state': state, 'year': year, 'price': price})

df = pd.DataFrame(rows)
df.head(10)

,state,year,price
0,AL,2010,265955
1,AL,2011,290122
2,AL,2012,286022
3,AL,2013,304213
4,AL,2014,304614
5,AL,2015,339975
6,AL,2016,364488
7,AL,2017,354905
8,AL,2018,402133
9,AL,2019,425096


In [8]:
import plotly.graph_objects as go

# Scrape state centroids from Google's public dataset
tables = pd.read_html('https://developers.google.com/public-data/docs/canonical/states_csv')
state_coords = tables[0]
state_coords.columns = ['state', 'latitude', 'longitude', 'name']

# Merge centroids into house price data
df = df.merge(state_coords[['state', 'latitude', 'longitude']], on='state')

# Create animated choropleth with year slider
fig = px.choropleth(
    df,
    locations='state',
    locationmode='USA-states',
    color='price',
    color_continuous_scale='Viridis',
    range_color=(150000, 800000),
    scope='usa',
    animation_frame='year',
    labels={'price': 'House Price ($)'},
    title='Average House Prices by State (2010–2024)'
)

# State label trace (uses first year's data for coordinates)
coords = df.drop_duplicates(subset='state')
label_trace = go.Scattergeo(
    locationmode='USA-states',
    lon=coords['longitude'],
    lat=coords['latitude'],
    text=coords['state'],
    mode='text',
    textfont=dict(size=8, color='white'),
    showlegend=False,
    hoverinfo='skip'
)

# Add labels to the base figure
fig.add_trace(label_trace)

# Add labels to every animation frame so they persist
for frame in fig.frames:
    frame.data = (*frame.data, label_trace)

fig.update_layout(margin={"r": 0, "t": 50, "l": 0, "b": 0})
fig.show()